In [2]:
#importing Packages
from bs4 import BeautifulSoup
import requests
import lxml.html as lh

!pip install geocoder
import geocoder
import pandas as pd
import numpy  as np
import json

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#map rendering library
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries Imported successfully')

    100% |████████████████████████████████| 102kB 2.9MB/s a 0:00:01
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: urllib3<1.23,>=1.21.1 in /opt/conda/

In [3]:
#linking URL to String url
url = 'https://www.mapsofindia.com/pincode/india/maharashtra/mumbai/'

#Scraping the webpage using requests libtrary
page =  requests.get(url).text

#Storing the contents of the website to doc variable
doc =  BeautifulSoup(page,'lxml')

In [4]:
doc_table = doc.find_all('table')

In [5]:
doc_table = doc_table[5]
fields = doc_table.find_all('td')
fields

[<td nowrap="" width="30%"><b>Location</b></td>,
 <td nowrap="" width="10%"><b>Pincode </b></td>,
 <td nowrap="" width="30%"><b>State </b></td>,
 <td nowrap="" width="30%"><b>District </b></td>,
 <td><a href="https://www.mapsofindia.com/pincode/india/maharashtra/mumbai/a-i-staff-colony.html">A I staff colony</a></td>,
 <td><b>400029 </b></td>,
 <td nowrap="">Maharashtra</td>,
 <td nowrap="">Mumbai </td>,
 <td><a href="https://www.mapsofindia.com/pincode/india/maharashtra/mumbai/aareymilk-colony.html">Aareymilk Colony</a></td>,
 <td><b>400065 </b></td>,
 <td nowrap="">Maharashtra</td>,
 <td nowrap="">Mumbai </td>,
 <td><a href="https://www.mapsofindia.com/pincode/india/maharashtra/mumbai/agripada.html">Agripada</a></td>,
 <td><b>400011 </b></td>,
 <td nowrap="">Maharashtra</td>,
 <td nowrap="">Mumbai </td>,
 <td><a href="https://www.mapsofindia.com/pincode/india/maharashtra/mumbai/airport.html">Airport</a></td>,
 <td><b>400099 </b></td>,
 <td nowrap="">Maharashtra</td>,
 <td nowrap="">M

In [6]:
#Intializinf the columns
location = []
pincode = []
state = []
district = []

#Appending Values into Columns
for item in range(0,len(fields),4):
    location.append(fields[item].text.strip())
    pincode.append(fields[item+1].text.strip())
    state.append(fields[item+2].text.strip())
    district.append(fields[item+3].text.strip())

#Creating the dataframe
df = pd.DataFrame(data=[location,pincode,state,district]).transpose()
df.columns = ['Location','Pincode','State','District']
df.head()

,Location,Pincode,State,District
0,Location,Pincode,State,District
1,A I staff colony,400029,Maharashtra,Mumbai
2,Aareymilk Colony,400065,Maharashtra,Mumbai
3,Agripada,400011,Maharashtra,Mumbai
4,Airport,400099,Maharashtra,Mumbai


In [7]:
df.drop(0,axis=0,inplace=True)

In [8]:
#Checking any null Values
df.isnull()

,Location,Pincode,State,District
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
5,False,False,False,False
6,False,False,False,False
7,False,False,False,False
8,False,False,False,False
9,False,False,False,False
10,False,False,False,False


In [9]:
#total rows and columns
df.shape
#looks like data frame doesn't contain any null values

(182, 4)

In [10]:
#Combining  the rows having same post codes
df_merge = df.groupby('Pincode')['Location'].apply(', '.join).reset_index()
df_merge.columns = ['Pincode', 'Location']
df_merge.head()

,Pincode,Location
0,400001,"Bazargate, M.P.t., Mumbai., Stock Exchange, Ta..."
1,400002,"Kalbadevi, Ramwadi, S. c. court, Shroff Mahaja..."
2,400003,"B P t colony, B.P.lane, Mandvi, Masjid, Noor B..."
3,400004,"Ambewadi, Charni Road, Chaupati, Girgaon, Madh..."
4,400005,"Asvini, Colaba, Holiday Camp, V.W.t.c."


In [11]:
df_merge.to_csv('Mumbai_Pincodes.csv')

In [106]:
#using geocoder:
APP_ID = 'APP_ID'
APP_CODE = 'APP_CODE'

pincode  = []
latitude = []
longitude = []
for elem in df_merge['Pincode']:
# initialize your variable to None
    lat_lng_coords = None

# looping until you get the coordinates
    while (lat_lng_coords is None):
        g = geocoder.here('{}, Mumbai, India'.format(elem), app_id = APP_ID, app_code = APP_CODE)
        lat_lng_coords = g.latlng
        print(lat_lng_coords)
    pincode.append(elem)
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
    print(elem, 'is RECEIVED.')

[18.94792, 72.84656]
400001 is RECEIVED.
[18.94788, 72.82618]
400002 is RECEIVED.
[18.95219, 72.83507]
400003 is RECEIVED.
[18.94958, 72.81907]
400004 is RECEIVED.
[18.90856, 72.81827]
400005 is RECEIVED.
[18.95394, 72.79954]
400006 is RECEIVED.
[18.96126, 72.81996]
400007 is RECEIVED.
[18.96732, 72.8274]
400008 is RECEIVED.
[18.95205, 72.84402]
400009 is RECEIVED.
[18.9715, 72.84411]
400010 is RECEIVED.
[18.98168, 72.82728]
400011 is RECEIVED.
[18.99797, 72.8416]
400012 is RECEIVED.
[18.99969, 72.82938]
400013 is RECEIVED.
[19.01605, 72.8479]
400014 is RECEIVED.
[18.99451, 72.85484]
400015 is RECEIVED.
[19.03546, 72.84138]
400016 is RECEIVED.
[19.04236, 72.85505]
400017 is RECEIVED.
[18.99538, 72.81688]
400018 is RECEIVED.
[19.02829, 72.8532]
400019 is RECEIVED.
[18.93438, 72.82475]
400020 is RECEIVED.
[18.92505, 72.82398]
400021 is RECEIVED.
[19.01321, 72.82684]
400025 is RECEIVED.
[18.97182, 72.80766]
400026 is RECEIVED.
[18.97861, 72.83484]
400027 is RECEIVED.
[19.02116, 72.83609]


In [107]:
#Creating Data Frame
df_latlan = pd.DataFrame(data = [pincode, latitude, longitude]).transpose()
df_latlan.columns = ['Pincode','Latitude', 'Longitude']
df_latlan.head()

,Pincode,Latitude,Longitude
0,400001,18.9479,72.8466
1,400002,18.9479,72.8262
2,400003,18.9522,72.8351
3,400004,18.9496,72.8191
4,400005,18.9086,72.8183


In [108]:
df_latlan.shape

(67, 3)

In [109]:
df_latlan.to_csv('Mumbai_Latlan.csv')

In [110]:
df_combine = pd.merge(df_merge, df_latlan, on = ['Pincode'], how = 'inner')
df_combine.head()

,Pincode,Location,Latitude,Longitude
0,400001,"Bazargate, M.P.t., Mumbai., Stock Exchange, Ta...",18.9479,72.8466
1,400002,"Kalbadevi, Ramwadi, S. c. court, Shroff Mahaja...",18.9479,72.8262
2,400003,"B P t colony, B.P.lane, Mandvi, Masjid, Noor B...",18.9522,72.8351
3,400004,"Ambewadi, Charni Road, Chaupati, Girgaon, Madh...",18.9496,72.8191
4,400005,"Asvini, Colaba, Holiday Camp, V.W.t.c.",18.9086,72.8183


In [111]:
df_combine

,Pincode,Location,Latitude,Longitude
0,400001,"Bazargate, M.P.t., Mumbai., Stock Exchange, Ta...",18.9479,72.8466
1,400002,"Kalbadevi, Ramwadi, S. c. court, Shroff Mahaja...",18.9479,72.8262
2,400003,"B P t colony, B.P.lane, Mandvi, Masjid, Noor B...",18.9522,72.8351
3,400004,"Ambewadi, Charni Road, Chaupati, Girgaon, Madh...",18.9496,72.8191
4,400005,"Asvini, Colaba, Holiday Camp, V.W.t.c.",18.9086,72.8183
5,400006,Malabar Hill,18.9539,72.7995
6,400007,"Bharat Nagar, Grant Road, N . s.patkar, S V ma...",18.9613,72.82
7,400008,"Falkland Road, J.J.hospital, Kamathipura, M A ...",18.9673,72.8274
8,400009,"Chinchbunder, Princess Dock",18.952,72.844
9,400010,"Dockyard Road, Mazgaon, Mazgaon Dock, Mazgaon ...",18.9715,72.8441


In [112]:
df_combine.to_csv('Mumbai_MainData.csv')

In [113]:
g_mum = geocoder.here('Mumbai',app_id=APP_ID,app_code=APP_CODE)
mum_lat = g_mum.lat
mum_lng = g_mum.lng

In [117]:
mumbai_maps = folium.Map(location=[mum_lat, mum_lng], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df_combine['Latitude'], df_combine['Longitude'], df_combine['Location']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#cc0000',
        fill_opacity=0.7,
        parse_html=False).add_to(mumbai_maps)  
    
mumbai_maps

In [118]:
CLIENT_ID = 'Client_ID'
CLIENT_SECRET = 'Client_secret'
VERSION = '20180605' # Foursquare API version

In [119]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [122]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500
mumbai_venues=getNearbyVenues(names=df_combine.Location, latitudes=df_combine.Latitude,
                    longitudes=df_combine.Longitude)

Bazargate, M.P.t., Mumbai., Stock Exchange, Tajmahal, Town Hall
Kalbadevi, Ramwadi, S. c. court, Shroff Mahajan, Thakurdwar
B P t colony, B.P.lane, Mandvi, Masjid, Noor Baug, Null Bazar, Parel Rly work shop
Ambewadi, Charni Road, Chaupati, Girgaon, Madhavbaug, Opera House
Asvini, Colaba, Holiday Camp, V.W.t.c.
Malabar Hill
Bharat Nagar, Grant Road, N . s.patkar, S V marg, Tardeo
Falkland Road, J.J.hospital, Kamathipura, M A marg, Mumbai Central
Chinchbunder, Princess Dock
Dockyard Road, Mazgaon, Mazgaon Dock, Mazgaon Road, V K bhavan
Agripada, Chinchpokli, Haines Road, Jacob Circle
Best Staff colony, Chamarbaug, Haffkin Institute, Lal Baug, Parel, Parel Naka
C G s colony, Delisle Road
Dadar, Dadar Colony, Naigaon
Sewri
Kapad Bazar, Mahim, Mahim Bazar, Mahim East, Mori Road
Dharavi, Dharavi Road
Worli, Worli Naka
Matunga Railway workshop
Central Building, Churchgate, Marine Lines
Nariman Point, New Yogakshema
New Prabhadevi road, Prabhadevi
Cumballa Hill, Cumballa Sea face, Dr Deshmukh 

In [156]:
mumbai_venues.shape

(968, 7)

In [159]:
mumbai_venues['Venue Category'].nunique()

159

In [126]:
mumbai_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Kalbadevi, Ramwadi, S. c. court, Shroff Mahaja...",18.94788,72.82618,Shree Thaker Bhojnalay,18.951217,72.828326,Indian Restaurant
1,"Kalbadevi, Ramwadi, S. c. court, Shroff Mahaja...",18.94788,72.82618,Parsi Dairy Farm,18.945378,72.828039,Cheese Shop
2,"Kalbadevi, Ramwadi, S. c. court, Shroff Mahaja...",18.94788,72.82618,Kyani & Co.,18.944062,72.828576,Café
3,"Kalbadevi, Ramwadi, S. c. court, Shroff Mahaja...",18.94788,72.82618,Marine Lines Railway Station,18.945704,72.823756,Train Station
4,"Kalbadevi, Ramwadi, S. c. court, Shroff Mahaja...",18.94788,72.82618,Cafe Coffee Day,18.945179,72.826334,Café


In [131]:
df_combine.head()

,Pincode,Location,Latitude,Longitude
0,400001,"Bazargate, M.P.t., Mumbai., Stock Exchange, Ta...",18.9479,72.8466
1,400002,"Kalbadevi, Ramwadi, S. c. court, Shroff Mahaja...",18.9479,72.8262
2,400003,"B P t colony, B.P.lane, Mandvi, Masjid, Noor B...",18.9522,72.8351
3,400004,"Ambewadi, Charni Road, Chaupati, Girgaon, Madh...",18.9496,72.8191
4,400005,"Asvini, Colaba, Holiday Camp, V.W.t.c.",18.9086,72.8183


In [128]:
len(mumbai_venues.Neighborhood.unique())

63

In [132]:
pd.concat([df_combine.Location, mumbai_venues.Neighborhood]).drop_duplicates(keep = False)

0     Bazargate, M.P.t., Mumbai., Stock Exchange, Ta...
45                                         Madh, Vesava
50    Borivali East, Daulat Nagar, Magthane, Rajendr...
60                           Malad East, Rani Sati marg
dtype: object

In [138]:
df_final = df_combine.drop([0,45,50,60],axis = 0)
len(df_final)

63

In [129]:
mumbai_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"A I staff colony, Santacruz P&t colony",4,4,4,4,4,4
"Aareymilk Colony, Nagari Niwara",7,7,7,7,7,7
"Agripada, Chinchpokli, Haines Road, Jacob Circle",6,6,6,6,6,6
"Airport, International Airport, Sahar P & t colony, Sahargaon",3,3,3,3,3,3
"Ambewadi, Charni Road, Chaupati, Girgaon, Madhavbaug, Opera House",10,10,10,10,10,10
"Andheri East, Nagardas Road",13,13,13,13,13,13
"Andheri Railway station, H.M.p. school",6,6,6,6,6,6
"Andheri, Azad Nagar",35,35,35,35,35,35
Antop Hill,3,3,3,3,3,3


In [140]:
# one hot encoding
mumbai_onehot = pd.get_dummies(mumbai_venues[['Venue Category']], prefix="", prefix_sep="")
mumbai_onehot.head()

,Airport,American Restaurant,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Garage,BBQ Joint,Bakery,...,Thai Restaurant,Theater,Trail,Train Station,Vegetarian / Vegan Restaurant,Volleyball Court,Wine Bar,Women's Store,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [150]:
# add neighborhood column back to dataframe
mumbai_onehot['Neighborhood'] = mumbai_venues['Neighborhood'] 

# move neighborhood column to the first column
cols = [mumbai_onehot.columns[-1]] + list(mumbai_onehot.columns[:-1])
mumbai_onehot = mumbai_onehot[cols]

#toronto_onehot = toronto_onehot.reindex(columns= cols)
mumbai_onehot.head()

,Neighborhood,Zoo,Airport,American Restaurant,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Garage,...,Tennis Court,Thai Restaurant,Theater,Trail,Train Station,Vegetarian / Vegan Restaurant,Volleyball Court,Wine Bar,Women's Store,Yoga Studio
0,"Kalbadevi, Ramwadi, S. c. court, Shroff Mahaja...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Kalbadevi, Ramwadi, S. c. court, Shroff Mahaja...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Kalbadevi, Ramwadi, S. c. court, Shroff Mahaja...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Kalbadevi, Ramwadi, S. c. court, Shroff Mahaja...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,"Kalbadevi, Ramwadi, S. c. court, Shroff Mahaja...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [151]:
mumbai_grouped = mumbai_onehot.groupby('Neighborhood').mean().reset_index()
mumbai_grouped.head()

,Neighborhood,Zoo,Airport,American Restaurant,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Garage,...,Tennis Court,Thai Restaurant,Theater,Trail,Train Station,Vegetarian / Vegan Restaurant,Volleyball Court,Wine Bar,Women's Store,Yoga Studio
0,"A I staff colony, Santacruz P&t colony",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.25,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Aareymilk Colony, Nagari Niwara",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Agripada, Chinchpokli, Haines Road, Jacob Circle",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Airport, International Airport, Sahar P & t co...",0.0,0.666667,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Ambewadi, Charni Road, Chaupati, Girgaon, Madh...",0.0,0.000000,0.0,0.1,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0


In [154]:
mumbai_grouped.shape

(63, 160)

In [155]:
# print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in mumbai_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = mumbai_grouped[mumbai_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----A I staff colony, Santacruz P&t colony----
                       venue  freq
0         Chinese Restaurant  0.50
1               Burger Joint  0.25
2           Asian Restaurant  0.25
3  Middle Eastern Restaurant  0.00
4          Mobile Phone Shop  0.00


----Aareymilk Colony, Nagari Niwara----
                  venue  freq
0                 Hotel  0.14
1                Resort  0.14
2  Gym / Fitness Center  0.14
3                  Café  0.14
4           Golf Course  0.14


----Agripada, Chinchpokli, Haines Road, Jacob Circle----
               venue  freq
0           Tea Room  0.17
1                Gym  0.17
2  Indian Restaurant  0.17
3        Coffee Shop  0.17
4         Restaurant  0.17


----Airport, International Airport, Sahar P & t colony, Sahargaon----
                       venue  freq
0                    Airport  0.67
1           Business Service  0.33
2                   Pharmacy  0.00
3         Mexican Restaurant  0.00
4  Middle Eastern Restaurant  0.00


----Ambewadi, Ch

In [160]:
#create the new dataframe and display the top 10 venues for each neighborhood.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [162]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = mumbai_grouped['Neighborhood']

for ind in np.arange(mumbai_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mumbai_grouped.iloc[ind, :], num_top_venues)


print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(63, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"A I staff colony, Santacruz P&t colony",Chinese Restaurant,Burger Joint,Asian Restaurant,Yoga Studio,Donut Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
1,"Aareymilk Colony, Nagari Niwara",Gym / Fitness Center,Farm,Golf Course,Hotel,Restaurant,Café,Resort,Diner,Farmers Market,Falafel Restaurant
2,"Agripada, Chinchpokli, Haines Road, Jacob Circle",Gym,Indian Restaurant,Coffee Shop,Restaurant,Tea Room,Racetrack,Farmers Market,Farm,Falafel Restaurant,Electronics Store
3,"Airport, International Airport, Sahar P & t co...",Airport,Business Service,Yoga Studio,Donut Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store
4,"Ambewadi, Charni Road, Chaupati, Girgaon, Madh...",Harbor / Marina,Restaurant,Gastropub,Indian Restaurant,Food Truck,Juice Bar,Pizza Place,Gym,Train Station,Aquarium


In [163]:
mumbai_grouped.head()

,Neighborhood,Zoo,Airport,American Restaurant,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Garage,...,Tennis Court,Thai Restaurant,Theater,Trail,Train Station,Vegetarian / Vegan Restaurant,Volleyball Court,Wine Bar,Women's Store,Yoga Studio
0,"A I staff colony, Santacruz P&t colony",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.25,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Aareymilk Colony, Nagari Niwara",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Agripada, Chinchpokli, Haines Road, Jacob Circle",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Airport, International Airport, Sahar P & t co...",0.0,0.666667,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Ambewadi, Charni Road, Chaupati, Girgaon, Madh...",0.0,0.000000,0.0,0.1,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0


In [180]:
# set number of clusters
kclusters = 10

mumbai_grouped_clustering = mumbai_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([4, 1, 9, 7, 1, 1, 9, 1, 1, 1, 3, 8, 1, 9, 3, 9, 3, 0, 1, 9, 1, 1, 3,
       1, 3, 1, 1, 9, 9, 1, 5, 3, 1, 9, 9, 1, 3, 9, 2, 9, 3, 1, 9, 1, 9, 6,
       9, 1, 1, 1, 1, 9, 1, 1, 9, 4, 1, 1, 2, 1, 3, 9, 1], dtype=int32)

In [181]:
df_final.head()

,Pincode,Location,Latitude,Longitude,Cluster Labels
1,400002,"Kalbadevi, Ramwadi, S. c. court, Shroff Mahaja...",18.9479,72.8262,1
2,400003,"B P t colony, B.P.lane, Mandvi, Masjid, Noor B...",18.9522,72.8351,3
3,400004,"Ambewadi, Charni Road, Chaupati, Girgaon, Madh...",18.9496,72.8191,3
4,400005,"Asvini, Colaba, Holiday Camp, V.W.t.c.",18.9086,72.8183,3
5,400006,Malabar Hill,18.9539,72.7995,3


In [182]:
mumbai_final = df_final

# add clustering labels
mumbai_final['Cluster Labels'] = kmeans.labels_

# merge mumbai_grouped with mumbai_final to add latitude/longitude for each neighborhood
mumbai_final = mumbai_final.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Location')
mumbai_final.head()

,Pincode,Location,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,400002,"Kalbadevi, Ramwadi, S. c. court, Shroff Mahaja...",18.9479,72.8262,4,Indian Restaurant,Café,Chinese Restaurant,Bar,Cheese Shop,Snack Place,Train Station,Department Store,Electronics Store,Field
2,400003,"B P t colony, B.P.lane, Mandvi, Masjid, Noor B...",18.9522,72.8351,1,Indian Restaurant,Convenience Store,Electronics Store,Restaurant,Rest Area,Café,Dessert Shop,Middle Eastern Restaurant,Chinese Restaurant,BBQ Joint
3,400004,"Ambewadi, Charni Road, Chaupati, Girgaon, Madh...",18.9496,72.8191,9,Harbor / Marina,Restaurant,Gastropub,Indian Restaurant,Food Truck,Juice Bar,Pizza Place,Gym,Train Station,Aquarium
4,400005,"Asvini, Colaba, Holiday Camp, V.W.t.c.",18.9086,72.8183,7,Park,Chinese Restaurant,Basketball Court,Restaurant,Department Store,Asian Restaurant,Ice Cream Shop,Fast Food Restaurant,Farmers Market,Farm
5,400006,Malabar Hill,18.9539,72.7995,1,Gym,Convenience Store,Restaurant,Dessert Shop,Park,Comfort Food Restaurant,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


In [184]:
# create map
map_clusters = folium.Map(location=[mum_lat, mum_lng], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mumbai_final['Latitude'], mumbai_final['Longitude'], mumbai_final['Location'], mumbai_final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
